## Conversational chatbot Demo
### rasa 2.0.0
1. conda create env --name rasa python=3.6
2. Rasa (pip install rasa==2.0.0)
3. SpaCy Language Model (pip install spacy==2.2.4)
4. nest_asyncio (pip install nest_asyncio)  -- needed to train in a Jupyter notebook

### rasa 3.2.6
1. conda create env --name rasa --clone base  (anaconda 2022.05 base has python 3.9.12)
2. pip install rasa (installed rasa==3.2.6)
3. pip install spacy (installed spacy==3.4.1)
4. pip install nest_asyncio  (needed to train in a Jupyter notebook)

### rasa 3.5.6
1. conda create env --name rasa3 --clone base  (anaconda 2022.05 base has python 3.9.12)
2. pip install rasa[spacy] (installed rasa 3.5.6  and spacy 3.4.4)
3. pip install nest_asyncio  (needed to train in a Jupyter notebook)

- https://rasa.com/docs/rasa/jupyter-notebooks/
- https://rasa.com/docs/rasa/tuning-your-model/
- https://rasa.com/blog/rasa-nlu-in-depth-part-1-intent-classification/
- https://rasa.com/blog/10-best-practices-for-designing-nlu-training-data/

In [ ]:
import nest_asyncio
nest_asyncio.apply()
print("Event loop ready.")

In [ ]:
import logging, io, json, warnings
# logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

In [3]:
import rasa
import rasa.nlu
import rasa.core
import spacy
print("rasa: {} spacy: {}".format(rasa.__version__, spacy.__version__))
print("rasa.nlu: {} rasa.core: {}".format(rasa.nlu.__version__, rasa.core.__version__))

print("Loading spaCy language model...")
print(spacy.load("en_core_web_md")("Hello world!"))

rasa: 3.5.6 spacy: 3.4.4
rasa.nlu: 3.5.6 rasa.core: 3.5.6
Loading spaCy language model...
Hello world!


In [4]:
!python --version

Python 3.9.7


In [5]:
# helper function for printing json data with proper indenting
def pprint(o):   
    print(json.dumps(o, indent=2))

### Location of 
* NLU training files - nlu.md, stories.md
* Config file - config.yml  (now the config file contains policies also..older versions of Rasa had a separate policy.yml file)
* Domain file - domain.yml

In [7]:
configfile = "config.yml"
training_files = "data/"
domainfile = "domain.yml"
outputdir = "models/"
print("config:",configfile)
print("nlu.yml,stories.yml:", training_files)
print("domain:", domainfile)
print("models:", outputdir)

config: config.yml
nlu.yml,stories.yml: data/
domain: domain.yml
models: models/


### Training a Dialogue (core) and NLU Model

In [8]:
print(domainfile, configfile, training_files, outputdir)

domain.yml config.yml data/ models/


In [9]:
import rasa
model_path = rasa.train(domainfile, configfile, [training_files], outputdir)
#model_path = rasa.train(domainfile, configfile, ['data/',['data/nlu.md', 'data/stories.md'], outputdir)
print(model_path)

The configuration for pipeline and policies was chosen automatically. It was written into the config file at 'config.yml'.
Your Rasa model is trained and saved at 'models\20230604-084403-beveled-deque.tar.gz'.
TrainingResult(model='models\\20230604-084403-beveled-deque.tar.gz', code=0, dry_run_results=None)


### Bot chat - simple method using the rasa.jupyter.chat module

In [ ]:
model_path

In [10]:
model_path[0]

'models\\20230604-084403-beveled-deque.tar.gz'

In [ ]:
# from rasa.jupyter import chat
# endpoints = "endpoints.yml"
# # chat(model_path, endpoints)
# chat(model_path)

### Bot chat - using the libraries - more flexible

In [11]:
import IPython
from IPython.display import clear_output
from rasa.core.agent import Agent
#from rasa.core.interpreter import NaturalLanguageInterpreter
from rasa.core.utils import EndpointConfig
import time

### Start Action server
In the project directory where actions.py is defined, run at command prompt: "rasa run actions" --  this will start the "action server" which will serve the actions at http://localhost:5055/webhook
* You can see the actions served at: http://localhost:5055/actions
* You can check the status of the actions server at: http://localhost:5055/health

In [12]:
global agent
#interpreter = NaturalLanguageInterpreter.create('models/nlu/default/current')
endpoint = EndpointConfig('http://localhost:5055/webhook')
#agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = endpoint)
# agent = Agent.load(model_path, action_endpoint = endpoint)
agent = Agent.load(model_path[0], action_endpoint = endpoint)
clear_output()

In [13]:
import asyncio
from typing import Text, List, Dict, Any

async def parse(text: Text):
    global agent
    response = await agent.handle_text(text)
    return response

def botResponse(userMsg):
    global agent
    if agent.is_ready():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        responses = loop.run_until_complete(parse(userMsg))

    if responses is not None :
        return responses
    else:
        return ['The response was not valid']


In [14]:
#messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
print("**********  Bot is ready to talk! Type your messages here or send 'stop' ******* ")
while True:
    a = input("user : ")
    if a == 'stop':
        break
    # responses = agent.handle_text(a)
    responses = botResponse(a)
    for response in responses:
        print("Bot :",response["text"])
        print()

# Tell me some news about fashion
print("Done...came out of bot")

**********  Bot is ready to talk! Type your messages here or send 'stop' ******* 


user :  Hello


Bot : Hello None, How are you doing?



user :  I am nikhil


Bot : Hi Nikhil. What news do you want to have today?



...

Bot : Hey Nikhil, how are you?



user :  I am fine, how are you doing today?


Bot : I'm doing great. Tell me How can I help you today?



user :  stop


Done...came out of bot


In [15]:
!rasa visualize

C:\Users\mayank\anaconda3\envs\rasa3\lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
2023-06-04 09:12:15 INFO     rasa.core.visualize  - Starting to visualize stories...

Processed story blocks: 100%|##########| 16/16 [00:00<00:00, 3200.08it/s, # trackers=1]
C:\Users\mayank\anaconda3\envs\rasa3\lib\site-packages\pkg_resources\__init__.py:1130: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  return get_provider(package_or_requirement).get_resource_filename(
2023-06-04 09:12:16 INFO     rasa.core.visualize  - Finished graph creation. Saved into file://C:\Edureka\course-AI-ML-PGP\05-Natural Language Processing\04-Building an Intent Based RASA 